# HOMEWORK 03

**Submitted by: Diego Sol**

CSCE 636-600: Deep Learning

Professor: Dr. Anxiao Jiang

## Task 1

**Check out the Jupyter notebook for Chapter 5 at https://github.com/fchollet/deep-learning-with-python-notebooks/blob/master/chapter05_fundamentals-of-ml.ipynb . Answer the question: what regularization techniques were mentioned in that Jupyter notebook? (5 points)**

## Task 2

**The MNIST dataset has 60,000 training images and 10,000 test images. Each image is a 28x28 array, where each array element is between 0 and 255. The images have 10 labels: 0, 1, 2, 3, 4, 5, 6, 7, 8, 9.**

**We now create a new dataset of 30,000 training images, 5,000 test images and 5 labels (which are 0, 1, 2, 3, 4) as follows. First, randomly pair up the training images of label 0 with the training images of label 1, to get 6,000 such pairs. Then, for each pair (A,B) where A is an image of label 0 and B is an image of label 1, we create a new image of size 28x28, where each element's value is the "average" of the two corresponding pixel values in A and B. (So the new image is a "mixture" of the two original images.) This way we create 6,000 new "mixture" images for training. In a similar way, we create 1,000 new "mixture" images for testing. We give all these 6,000+1,000=7,000 new "mixture" images the new label 0. Then, in the same way, we create 6,000 new training images and 1,000 new test images by mixing the original images of label "2" and "3", and give them the new label 1; create 6,000 new training images and 1,000 new test images by mixing the original images of label "4" and "5", and give them the new label 2; create 6,000 new training images and 1,000 new test images by mixing the original images of label "6" and "7", and give them the new label 3; create 6,000 new training images and 1,000 new test images by mixing the original images of label "8" and "9", and give them the new label 4.**   

**Your task: submit your code that creates the above new dataset; then for each of the 5 new labels, randomly select 2 images of that label from your new dataset, and display them in your submitted Jupyter notebook. (5 points)**

## Task 3

**Design a neural network model to classify the 5 classes in the new dataset, and optimize it by tuning its hyper-parameters and trying our learned regualization techniques (such as L1 regularization, L2 regularization, dropout).**

**Your task: For your final (namely, optimized) neural network model, submit its code, and show the model's performance (including loss value and accuracy) for training, validation and testing. (For training performance and validation performance, illustrate them using figures, where the x-axis is the number of training epochs, and the y-axis is the loss or accuracy. For testing performance, just show the values of loss and accuracy.) Also, answer the questions: in this process of optimizing your model, did you observe underfitting? Did you observe overfitting? Did you try the reguliazation techniques? Did they help? (10 points)**